In [1]:
import os
import requests
import re
from datetime import datetime, timezone
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pyperclip
import time

#git add . && git commit -m "initial commit" && git push origin main

endpoint = "https://active-monitor-48.hasura.app/v1/graphql"
admin_key = "bAQuK7HSYvMAp6S6pnqXH0wQlyuKNUICzoW3jwecc27pwz6COLhE750s5YAec7Hz"

def query_hasura_graphql(endpoint, admin_key, query, variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }

    data = {
        'query': query,
        'variables': variables
    }
    response = requests.post(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)
        
        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False
        
def mutation_hasura_graphql(endpoint, admin_key, mutation_query, mutation_variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }
    response = requests.post(endpoint, json={'query': mutation_query, 'variables': mutation_variables}, headers=headers)
    if response.ok:
        data = response.json()
        print(data)
        return True, data
    else:
        print(f"Mutation failed with status code {response.status_code}: {response.text}")
        return False, None

def update_article_details(offset1):
    graphql_query = '''
    query MyQuery($limit: Int!, $offset: Int!) {
    synopse_articles_t_v1_rss1_articles(limit: $limit, offset: $offset, where: {is_in_detail: {_eq: 0}}, order_by: {post_published: desc}) {
        post_link
        is_default_image
        id
        }
    }
    '''
    offset = offset1
    mutation_query = """
    mutation MyMutation($objects: [synopse_articles_t_v1_rss1_articles_detail_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}) {
        insert_synopse_articles_t_v1_rss1_articles_detail(objects: $objects, on_conflict: {constraint: t_v1_rss1_articles_detail_article_id_key}) {
            affected_rows
        }
        update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
            affected_rows
        }
        }

    """    
    options = webdriver.EdgeOptions()
    options.use_chromium = True
    options.add_experimental_option("prefs", {
        "profile.default_content_setting_values.notifications": 2,  # Disable notifications
        "profile.default_content_setting_values.media_stream_mic": 2,  # Disable microphone
        "profile.default_content_setting_values.media_stream_camera": 2,  # Disable camera
        "profile.default_content_setting_values.geolocation": 2,  # Disable location services
        "profile.default_content_setting_values.automatic_downloads": 2  # Disable automatic downloads
    })
    options.add_argument("--no-first-run")  # Skip the welcome page
    options.add_argument("--disable-infobars")  # Disable the info bars
    options.add_argument("--disable-extensions")  # Disable extensions
    options.add_argument("--disable-popup-blocking")  # Disable popups
    options.add_argument("--disable-features=EdgeTips") 
    options.page_load_strategy = 'eager'
    options.add_argument('--enable-immersive-reader')
    driver = webdriver.Edge(options=options)
    while True:
        variables = {
        "limit": 2,
        "offset": offset1
        }
        response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
        #print(variables, response_data)
        #print(response_data)
        post_links_array = []
        ids=[]
        is_default_image_array = []
        if response_data:
            post_links_array = [item["post_link"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
            is_default_image_array = [item["is_default_image"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
            ids=[item["id"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
        articles_detail = []
        articles_update = []
        if len(post_links_array) == 0:
            break
        try:
            for a in range(len(post_links_array)):
                main_link = post_links_array[a]
                print(main_link)
                driver.get(main_link)
                time.sleep(2)
                get_url = driver.current_url
                read_link= "read://aqq"+get_url
                driver.get(read_link)
                time.sleep(2)
                ActionChains(driver).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()
                ActionChains(driver).key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
                text = pyperclip.paste()
                text2 = text
                text3 = text2.split('\n')
                text3 = [s.replace('\r', '') for s in text3]
                special_chars = set("!@#$%^&*()_+[]{}|;:'\",<>?")
                text4 = [s for s in text3 if len(s) > 0 and (s[0] not in special_chars or s[-1] not in special_chars)]
                my_list = text4
                if ' '.join(my_list) == "Hmmm… can't reach this page":
                    offset = offset + 1
                    break
                my_set = set()
                desription = []
                for item in my_list:
                    if item not in my_set:
                        desription.append(item)
                        my_set.add(item)
                #print(desription)
                images_final = []
                articles_detail.append({
                    "article_id": ids[a],
                    "title": desription[0],
                    "description": desription[1:],
                    "image_link": images_final,
                }
                )
                if (is_default_image_array[a] == 0 and len(images_final) > 0):
                    articles_update.append({
                        "where": {"post_link" : { "_eq": main_link }},
                        "_set": {"is_in_detail": 1 , "image_link": images_final[0], "is_default_image": 1}
                    })
                else:
                    articles_update.append({
                        "where": {"post_link" : { "_eq": main_link }},
                        "_set": {"is_in_detail": 1}
                    })
                
                #print(main_link, desription[0], desription[1:], images_final)
            #print(articles_update)
            mutation_variables = {
            "objects": articles_detail,
            "updates": articles_update,
            }
            out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
            break
        except:
            offset = offset + 1
            mutation_variables = {
            "objects": articles_detail,
            "updates": articles_update,
            }
            out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
    driver.quit() 


In [2]:
update_article_details(100)

https://www.thehindu.com/news/international/us-venezuela-swap-prisoners-maduro-ally-for-10-americans-plus-fugitive-contractor-fat-leonard/article67662305.ece
["Hmmm… can't reach this pageIt looks like the webpage at read://aqqhttps//www.thehindu.com/news/international/us-venezuela-swap-prisoners-maduro-ally-for-10-americans-plus-fugitive-contractor-fat-leonard/article67662305.ece might be having issues or it may have moved permanently to a new web address.", 'ERR_FAILED']
https://www.thehindu.com/news/international/un-says-up-to-300000-sudanese-fled-their-homes-after-a-notorious-group-seized-their-safe-haven/article67662367.ece
{'data': {'insert_synopse_articles_t_v1_rss1_articles_detail': {'affected_rows': 1}, 'update_synopse_articles_t_v1_rss1_articles_many': [{'affected_rows': 1}]}}
https://www.thehindu.com/news/international/un-says-up-to-300000-sudanese-fled-their-homes-after-a-notorious-group-seized-their-safe-haven/article67662367.ece
["Hmmm… can't reach this pageIt looks like t

In [2]:
t1 = ["Hmmm… can't reach this pageIt looks like the webpage at read://aqqhttps//www.thehindu.com/news/international/us-venezuela-swap-prisoners-maduro-ally-for-10-americans-plus-fugitive-contractor-fat-leonard/article67662305.ece might be having issues or it may have moved permanently to a new web address.", 'ERR_FAILED']

if t1[0].startswith("Hmmm… can't reach this pageIt"):
    print("The string starts with 'Hmmm… can't reach this pageIt'")


The string starts with 'Hmmm… can't reach this pageIt'


In [1]:
graphql_query = '''
query MyQuery($limit: Int!, $offset: Int!) {
synopse_articles_t_v1_rss1_articles(limit: $limit, offset: $offset, where: {is_in_detail: {_eq: 0}}, order_by: {post_published: desc}) {
    post_link
    is_default_image
    id
    }
}
'''
mutation_query = """
mutation MyMutation($objects: [synopse_articles_t_v1_rss1_articles_detail_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}) {
    insert_synopse_articles_t_v1_rss1_articles_detail(objects: $objects, on_conflict: {constraint: t_v1_rss1_articles_detail_article_id_key}) {
        affected_rows
    }
    update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
        affected_rows
    }
    }

"""    
options = webdriver.EdgeOptions()
options.use_chromium = True
options.page_load_strategy = 'eager'
options.add_argument('--enable-immersive-reader')
driver = webdriver.Edge(options=options)
variables = {
"limit": 8,
"offset": 0
}
response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
#print(variables, response_data)
#print(response_data)
post_links_array = []
ids=[]
is_default_image_array = []
if response_data:
    post_links_array = [item["post_link"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
    is_default_image_array = [item["is_default_image"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
    ids=[item["id"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
articles_detail = []
articles_update = []
for a in range(len(post_links_array)):
    main_link = post_links_array[a]
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[a + 1])
    driver.get(main_link)
for a in range(len(post_links_array)):
    driver.switch_to.window(driver.window_handles[a + 1])
    get_url = driver.current_url
    read_link= "read://"+get_url
    driver.get(read_link)

NameError: name 'webdriver' is not defined

In [8]:
import pyautogui

def mousemove():
    start_x, start_y = 90, 90
    side_length = 100
    pyautogui.moveTo(start_x, start_y)
    pyautogui.moveTo(start_x + side_length, start_y)
    pyautogui.moveTo(start_x + side_length, start_y + side_length)
    pyautogui.moveTo(start_x, start_y + side_length)
    pyautogui.moveTo(start_x, start_y)

In [ ]:
import os
import requests
import re
from datetime import datetime, timezone
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pyperclip
import multiprocessing
import time
import psutil
import shutil
import pyautogui

#git add . && git commit -m "initial commit" && git push origin main

endpoint = "https://active-monitor-48.hasura.app/v1/graphql"
admin_key = "bAQuK7HSYvMAp6S6pnqXH0wQlyuKNUICzoW3jwecc27pwz6COLhE750s5YAec7Hz"


def mousemove():
    start_x, start_y = 90, 90
    side_length = 100
    pyautogui.moveTo(start_x, start_y)
    pyautogui.moveTo(start_x + side_length, start_y)
    pyautogui.moveTo(start_x + side_length, start_y + side_length)
    pyautogui.moveTo(start_x, start_y + side_length)
    pyautogui.moveTo(start_x, start_y)
    
def delete_tmp():
    rootDir = '/tmp'
    for dirName, subdirList, fileList in os.walk(rootDir):
        if dirName.startswith(rootDir + "/.com.microsoft"):
            try:
                shutil.rmtree(dirName)
            except FileNotFoundError:
                print(f"No directories found starting with '.com.microsoft'")
                break
	
def query_hasura_graphql(endpoint, admin_key, query, variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }

    data = {
        'query': query,
        'variables': variables
    }
    response = requests.post(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)
        
        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False
        
def mutation_hasura_graphql(endpoint, admin_key, mutation_query, mutation_variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }
    response = requests.post(endpoint, json={'query': mutation_query, 'variables': mutation_variables}, headers=headers)
    if response.ok:
        data = response.json()
        print(data)
        return True, data
    else:
        print(f"Mutation failed with status code {response.status_code}: {response.text}")
        return False, None

def update_article_details1(offset):
    graphql_query = '''
    query MyQuery($limit: Int!, $offset: Int!) {
    synopse_articles_t_v1_rss1_articles(limit: $limit, offset: $offset, where: {is_in_detail: {_eq: 0}}, order_by: {post_published: desc}) {
        post_link
        is_default_image
        id
        }
    }
    '''
    offset1 = offset
    mutation_query = """
    mutation MyMutation($objects: [synopse_articles_t_v1_rss1_articles_detail_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}) {
        insert_synopse_articles_t_v1_rss1_articles_detail(objects: $objects, on_conflict: {constraint: t_v1_rss1_articles_detail_article_id_key}) {
            affected_rows
        }
        update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
            affected_rows
        }
        }

    """    
    options = webdriver.EdgeOptions()
    options.use_chromium = True
    options.add_experimental_option("prefs", {
        "profile.default_content_setting_values.notifications": 2,  # Disable notifications
        "profile.default_content_setting_values.media_stream_mic": 2,  # Disable microphone
        "profile.default_content_setting_values.media_stream_camera": 2,  # Disable camera
        "profile.default_content_setting_values.geolocation": 2,  # Disable location services
        "profile.default_content_setting_values.automatic_downloads": 2  # Disable automatic downloads
    })
    options.add_argument("--no-first-run")  # Skip the welcome page
    options.add_argument("--disable-infobars")  # Disable the info bars
    options.add_argument("--disable-extensions")  # Disable extensions
    options.add_argument("--disable-popup-blocking")  # Disable popups
    options.add_argument("--disable-features=EdgeTips") 
    options.page_load_strategy = 'eager'
    options.add_argument('--enable-immersive-reader')
    driver = webdriver.Edge(options=options)
    variables = {
    "limit": 1,
    "offset": offset1
    }
    response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
    #print(variables, response_data)
    #print(response_data)
    post_links_array = []
    ids=[]
    is_default_image_array = []
    if response_data:
        post_links_array = [item["post_link"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
        is_default_image_array = [item["is_default_image"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
        ids=[item["id"] for item in response_data["data"]["synopse_articles_t_v1_rss1_articles"]]
    articles_detail = []
    articles_update = []
    for a in range(len(post_links_array)):
        main_link = post_links_array[a]
        print(main_link)
        driver.get(main_link)
        time.sleep(2)
        get_url = driver.current_url
        read_link= "read://"+get_url
        driver.get(read_link)
        time.sleep(2)
        ActionChains(driver).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()
        ActionChains(driver).key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
        text = pyperclip.paste()
        text2 = text
        text3 = text2.split('\n')
        text3 = [s.replace('\r', '') for s in text3]
        special_chars = set("!@#$%^&*()_+[]{}|;:'\",<>?")
        text4 = [s for s in text3 if len(s) > 0 and (s[0] not in special_chars or s[-1] not in special_chars)]
        my_list = text4
        if my_list[0] == "Hmmm… can't reach this page":
            articles_update.append({
                "where": {"post_link" : { "_eq": main_link }},
                "_set": {"is_in_detail": 2}
            })
        my_set = set()
        desription = []
        for item in my_list:
            if item not in my_set:
                desription.append(item)
                my_set.add(item)
        #print(desription)
        images_final = []
        articles_detail.append({
            "article_id": ids[a],
            "title": desription[0],
            "description": desription[1:],
            "image_link": images_final,
        }
        )
        if (is_default_image_array[a] == 0 and len(images_final) > 0):
            articles_update.append({
                "where": {"post_link" : { "_eq": main_link }},
                "_set": {"is_in_detail": 1 , "image_link": images_final[0], "is_default_image": 1}
            })
        else:
            articles_update.append({
                "where": {"post_link" : { "_eq": main_link }},
                "_set": {"is_in_detail": 1}
            })
        
        #print(main_link, desription[0], desription[1:], images_final)
    #print(articles_update)
    mutation_variables = {
    "objects": articles_detail,
    "updates": articles_update,
    }
    out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)
    driver.quit()

def kill_if_too_long(function, args, max_execution_time):
    # Create a process that runs the function with the given arguments
    process = multiprocessing.Process(target=function, args=args)
    process.start()
    mousemove()
    delete_tmp()
    # Wait for the specified maximum execution time
    process.join(timeout=max_execution_time)

    # If the process is still running after the maximum execution time, terminate it
    if process.is_alive():
        print("Function is running too long, terminating...")
        process.terminate()
        process.join()

def kill_process_tree(process_name):
    for proc in psutil.process_iter(['pid', 'name']):
        # Check if the process name starts with the given name string.
        if proc.info['name'].startswith(process_name):
            # Kill the process tree
            parent = psutil.Process(proc.info['pid'])
            for child in parent.children(recursive=True):
                try:
                    child.kill()
                except psutil.NoSuchProcess:
                    pass
            try:
                parent.kill()
            except psutil.NoSuchProcess:
                pass

def get_process_names():
    process_names = [proc.info['name'] for proc in psutil.process_iter(['name'])]
    return process_names

if __name__ == "__main__":
    while True:
        try:
            kill_if_too_long(update_article_details1, (30,), 30)
            kill_process_tree('msedge')
        except Exception as e:
            print(f"An error occurred: {e}")




In [1]:
%%file grouping_l1.py

import os
import requests
import feedparser
import re
import numpy
from datetime import datetime, timezone
from multiprocessing import Process, set_start_method
import multiprocessing

endpoint = "https://active-monitor-48.hasura.app/v1/graphql"
admin_key = "bAQuK7HSYvMAp6S6pnqXH0wQlyuKNUICzoW3jwecc27pwz6COLhE750s5YAec7Hz"

def query_hasura_graphql(endpoint, admin_key, query, variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }

    data = {
        'query': query,
        'variables': variables
    }
    response = requests.post(endpoint, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status code {response.status_code}")
        return None

def is_valid_timezone_format(published):
    try:
        # Attempt to parse the string
        date_format = "%a, %d %b %Y %H:%M:%S %z"
        date_object = datetime.strptime(published, date_format)

        hasura_timestamp = date_object.astimezone(timezone.utc).isoformat()
        return True, hasura_timestamp
    except ValueError:
        # If parsing fails, the string is not in the correct format
        return False, None

def check_date_format(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%dT%H:%M:%S%z')
        return True
    except ValueError:
        return False

def mutation_hasura_graphql(endpoint, admin_key, mutation_query, mutation_variables):
    headers = {
        'Content-Type': 'application/json',
        'x-hasura-admin-secret': f'{admin_key}'
    }
    response = requests.post(endpoint, json={'query': mutation_query, 'variables': mutation_variables}, headers=headers)
    if response.ok:
        data = response.json()
        print(data)
        return True, data
    else:
        print(f"Mutation failed with status code {response.status_code}: {response.text}")
        return False, None

def grouping_l1(offset1):
    print(offset1)
    graphql_query = '''
    query MyQuery($offset: Int!, $limit: Int!) {
      synopse_articles_t_v1_rss1_articles(offset: $offset, limit: $limit, order_by: {created_at: desc}, where: {is_grouped: {_eq: 0}, is_vectorized: {_eq: 1}}) {
        id
      }
    }
    '''
    offset = offset1
    mutation_query = """
    mutation MyMutation($objects: [synopse_articles_t_v3_article_groups_l1_insert_input!] = {}, $updates: [synopse_articles_t_v1_rss1_articles_updates!] = {where: {}}) {
      insert_synopse_articles_t_v3_article_groups_l1(objects: $objects, on_conflict: {constraint: t_v3_article_groups_l1_article_id_key}) {
        affected_rows
      }
      update_synopse_articles_t_v1_rss1_articles_many(updates: $updates) {
        affected_rows
      }
    }
    """
    func_query = '''
    query MyQuery($p_article_id: bigint!) {
      synopse_articles_f_get_similar_articles_l1(args: {p_article_id: $p_article_id}) {
        article_id
      }
    }
    '''
    while True:
        variables = {
        "limit": 10,
        "offset": offset
        }
        synopse_articles_t_v3_article_groups_l1_insert_input_loc=[]
        synopse_articles_t_v1_rss1_articles_updates_loc=[]
        response_data = query_hasura_graphql(endpoint, admin_key, graphql_query, variables)
        if len(response_data['data']['synopse_articles_t_v1_rss1_articles']) == 0:
            break
        s1= []
        ids=[]
        for response in response_data['data']['synopse_articles_t_v1_rss1_articles']:
            func_variables = {
                "p_article_id": response['id']
                }
            func_response_data = query_hasura_graphql(endpoint, admin_key, func_query, func_variables)
            article_group = []
            #print(json.dumps(func_response_data, indent=4))
            if len(func_response_data['data']['synopse_articles_f_get_similar_articles_l1']) > 0:
                for func_response in func_response_data['data']['synopse_articles_f_get_similar_articles_l1']:
                    article_group.append(func_response['article_id'])

            synopse_articles_t_v3_article_groups_l1_insert_input_loc.append({
                "article_id": response['id'],
                "initial_group": article_group,
                "article_count": len(article_group)
                }
                )
            synopse_articles_t_v1_rss1_articles_updates_loc.append({
                "where": {"id" : { "_eq": response['id'] }},
                "_set": {"is_grouped": 1}
                })
        mutation_variables = {
        "objects": synopse_articles_t_v3_article_groups_l1_insert_input_loc,
        "updates": synopse_articles_t_v1_rss1_articles_updates_loc,
        }
        out1 = mutation_hasura_graphql(endpoint=endpoint, admin_key=admin_key, mutation_query=mutation_query, mutation_variables=mutation_variables)


if __name__ == "__main__":
    # Define the list of arguments
    n1 =  2
    mod = 50
    n2 = 10
    offset = (mod * (n2 + 1)) * n1
    set_start_method('spawn', force=True)
    args = []
    for i in range(0 , n2):
      args.append((i * mod) + offset)
    print(args)

    # Create a list to hold the processes
    processes = []

    # Create and start a process for each argument
    for arg in args:
        process = multiprocessing.Process(target=grouping_l1, args=(arg,))
        processes.append(process)
        process.start()

    # Wait for all processes to finish
    for process in processes:
        process.join()

Writing grouping_l1.py


In [2]:
import sys
!{sys.executable} grouping_l1.py